In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("normal_phone.csv")
data

,index,name,price,number_of_comment,pin(mAh),brand,weight(g),screen_size(inch),comment
0,0,Điện Thoại Nokia 105 Dual Sim (2019) - Hàng Ch...,350000,324,800.0,Nokia,73.0,1.77,1 Về máy Mới khui hộp và dùng chưa biết dài lâ...
1,0,Điện Thoại Nokia 105 Dual Sim (2019) - Hàng Ch...,350000,324,800.0,Nokia,73.0,1.77,Ko Nên Mua bà con ơi Máy bị Lỗi phần hiển th...
2,0,Điện Thoại Nokia 105 Dual Sim (2019) - Hàng Ch...,350000,324,800.0,Nokia,73.0,1.77,Mình đã mua Nokia 105 2019 trên Tiki và chọn g...
3,0,Điện Thoại Nokia 105 Dual Sim (2019) - Hàng Ch...,350000,324,800.0,Nokia,73.0,1.77,Ti ki giao hàng khá nhanh sớm hơn 01 ngày so v...
4,0,Điện Thoại Nokia 105 Dual Sim (2019) - Hàng Ch...,350000,324,800.0,Nokia,73.0,1.77,mình mua về cắm sim phụ nhận opt là chủ yếu hô...
...,...,...,...,...,...,...,...,...,...
2655,102,Điện Thoại Di Động GSM Zono N105 - Hàng Nhập K...,349000,13,1000.0,Zono,NaN,1.80,giao hàng nhanh đóng gói chuyên nghiệp mình mu...
2656,102,Điện Thoại Di Động GSM Zono N105 - Hàng Nhập K...,349000,13,1000.0,Zono,NaN,1.80,Máy nhìn rất chất lượng phím bấm rất êm nghe g...
2657,102,Điện Thoại Di Động GSM Zono N105 - Hàng Nhập K...,349000,13,1000.0,Zono,NaN,1.80,Bền minh làm ăn chán wa e đặt 1cai điện thoại ...
2658,102,Điện Thoại Di Động GSM Zono N105 - Hàng Nhập K...,349000,13,1000.0,Zono,NaN,1.80,chả biết máy các bác mua xài thế nào chứ máy e...


In [3]:
review_predict = data['comment']

In [4]:
sentiment = pd.read_csv('text.csv',index_col=[0])
sentiment

,text_lb,text
2,1.0,"Ấn tượng, tôi sẽ quay lại vào dịp nghỉ lần sau"
3,1.0,"Quá tuyệt vời , view đẹp , chủ nhà thân thiện ..."
6,1.0,"Phòng nghỉ đáng giá, tiện nghi cơ bản đầy đủ ^..."
7,1.0,Thuận tiện cho việc đi thăm thú cảnh quan.\nBạ...
10,1.0,Nhân viên thân thiện
...,...,...
6994,1.0,"Anh chủ rất là nhiệt tình và vui vẻ, phòng sạc..."
6995,1.0,"Vị trí đẹp, mặc dù chúng tôi là khách duy nhất..."
6996,-1.0,"Không có chỗ đỗ xe, phải gửi xe ở ngoài rất tố..."
6997,1.0,"Gửi quản lý cấp cao của khách sạn, Rất tiếc hì..."


In [5]:
with open('vietnamese_stopwords.txt', encoding="utf8") as f:
    stopwords = []
    for line in f:
        stopwords.append("_".join(line.strip().split()))

In [6]:
import re
from pyvi import ViTokenizer
def preprocessor(text):
    corpus = []
    for i in range(0, len(text)):
        review = re.sub(r"http\S+", "", str(text[i]))
        review = re.sub(r"#\S+", "", review)
        review = re.sub(r"@\S+", "", review)
        review = re.sub('[_]',' ',review)
        review = re.sub('[^a-zA-Z_áàạảãăắằặẵẳâấầẩậẫđíỉìịĩóòỏọõôốồổộỗơớờởợỡéèẹẽẻêếềểệễúùủũụưứừửựữýỳỷỹỵÁÀẢÃẠĂẮẰẲẲẶẴÂẤẦẬẪẨĐÍÌỈỊĨÓÒỎỌÕÔỐỒỔỘỖƠỚỜỞỢỠÉÈẺẸẼÊẾỀỆỂỄÚÙỦŨỤƯỨỪỬỰỮÝỲỶỴỸ]',
                        ' ',review)
        review = ViTokenizer.tokenize(review)
        review = review.lower()
        review = review.split()
        review = [word for word in review if not word in  set(stopwords)]
        review = ' '.join(review)
        corpus.append(review)
    return corpus

In [7]:
X = sentiment['text'].values
corpus = preprocessor(X)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(corpus) 
X_data_tfidf =  tfidf_vect.transform(corpus)

from sklearn.model_selection import train_test_split
X = X_data_tfidf
y = sentiment['text_lb'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=102)

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0)
clf.fit(X_train, y_train)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
predictions = clf.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))

accuracy: 0.8759615384615385


In [9]:
from sklearn import svm
classifier = svm.SVC(probability=True)
classifier.fit(X_train, y_train)
train_predictions = classifier.predict(X_train)
predictions = classifier.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))
print('confusion matrix:\n',confusion_matrix(y_test,predictions))
print('classification report:\n',classification_report(y_test,predictions))

accuracy: 0.885576923076923
confusion matrix:
 [[ 84 100]
 [ 19 837]]
classification report:
               precision    recall  f1-score   support

        -1.0       0.82      0.46      0.59       184
         1.0       0.89      0.98      0.93       856

    accuracy                           0.89      1040
   macro avg       0.85      0.72      0.76      1040
weighted avg       0.88      0.89      0.87      1040



In [10]:
X_1 = review_predict.values
corpus1 = preprocessor(X_1)
X_data_tfidf_1 =  tfidf_vect.transform(corpus1)

In [11]:
proba = classifier.predict_proba(X_data_tfidf_1)

In [12]:
df1 = pd.DataFrame(data=proba, columns=['Negative', 'Positive'])

In [14]:
data = pd.concat([data, df1], axis = 1).drop(columns = ['comment'])
data

,index,name,price,number_of_comment,pin(mAh),brand,weight(g),screen_size(inch),Negative,Positive
0,0,Điện Thoại Nokia 105 Dual Sim (2019) - Hàng Ch...,350000,324,800.0,Nokia,73.0,1.77,0.733013,0.266987
1,0,Điện Thoại Nokia 105 Dual Sim (2019) - Hàng Ch...,350000,324,800.0,Nokia,73.0,1.77,0.247503,0.752497
2,0,Điện Thoại Nokia 105 Dual Sim (2019) - Hàng Ch...,350000,324,800.0,Nokia,73.0,1.77,0.243498,0.756502
3,0,Điện Thoại Nokia 105 Dual Sim (2019) - Hàng Ch...,350000,324,800.0,Nokia,73.0,1.77,0.421099,0.578901
4,0,Điện Thoại Nokia 105 Dual Sim (2019) - Hàng Ch...,350000,324,800.0,Nokia,73.0,1.77,0.569517,0.430483
...,...,...,...,...,...,...,...,...,...,...
2655,102,Điện Thoại Di Động GSM Zono N105 - Hàng Nhập K...,349000,13,1000.0,Zono,NaN,1.80,0.279993,0.720007
2656,102,Điện Thoại Di Động GSM Zono N105 - Hàng Nhập K...,349000,13,1000.0,Zono,NaN,1.80,0.158794,0.841206
2657,102,Điện Thoại Di Động GSM Zono N105 - Hàng Nhập K...,349000,13,1000.0,Zono,NaN,1.80,0.801908,0.198092
2658,102,Điện Thoại Di Động GSM Zono N105 - Hàng Nhập K...,349000,13,1000.0,Zono,NaN,1.80,0.701917,0.298083


In [15]:
group_data = data.groupby(['index', 'name', 'price', 'number_of_comment', 'pin(mAh)', 'brand', 'weight(g)', 'screen_size(inch)'], as_index=False, dropna=False).mean()
group_data['rank_point'] = group_data.number_of_comment * group_data.Positive
group_data

,index,name,price,number_of_comment,pin(mAh),brand,weight(g),screen_size(inch),Negative,Positive,rank_point
0,0,Điện Thoại Nokia 105 Dual Sim (2019) - Hàng Ch...,350000,324,800.0,Nokia,73.0,1.77,0.191413,0.808587,261.982173
1,1,Điện Thoại Nokia 105 Single Sim - Hàng Chính H...,340000,280,800.0,Nokia,73.0,1.77,0.205022,0.794978,222.593702
2,2,Điện Thoại Nokia 150 Dual Sim - Hàng Chính Hãng,599000,34,1020.0,Nokia,100.0,2.40,0.298605,0.701395,23.847440
3,3,Điện Thoại Nokia 150 (2020) - Hàng Chính Hãng,590000,128,1020.0,Nokia,90.0,2.40,0.179362,0.820638,105.041692
4,4,Điện Thoại Nokia 6300 4G - Hàng Chính Hãng,1150000,25,1500.0,Nokia,NaN,2.40,0.300685,0.699315,17.482873
...,...,...,...,...,...,...,...,...,...,...,...
85,95,Điện Thoại Bavapen B52 - Hàng Chính Hãng,430000,8,3500.0,Bavapen,110.0,2.80,0.320946,0.679054,5.432429
86,96,Điện Thoại Di Động Forme L6 - Hàng Chính Hãng,345000,8,1500.0,Forme,NaN,NaN,0.230877,0.769123,6.152981
87,97,Điện thoại Masstel Fami 12 - Hàng Chính Hãng,389000,1,1000.0,Masstel,NaN,NaN,0.803772,0.196228,0.196228
88,101,Điện thoại Forme L03 màn hình 2.4inch pin 35...,360000,1,3500.0,Forme,250.0,NaN,0.642542,0.357458,0.357458


In [16]:
rank_data = group_data.drop(columns=['index', 'name', 'brand', 'number_of_comment', 'Negative', 'Positive'])

In [18]:
pd.set_option('display.max_rows', rank_data.shape[0]+1)
rank_data[lambda col :rank_data.columns] = rank_data[lambda col :rank_data.columns].replace(r'\D+', np.NaN, regex=True)
rank_data

,price,pin(mAh),weight(g),screen_size(inch),rank_point
0,350000,800.0,73.0,1.77,261.982173
1,340000,800.0,73.0,1.77,222.593702
2,599000,1020.0,100.0,2.40,23.847440
3,590000,1020.0,90.0,2.40,105.041692
4,1150000,1500.0,NaN,2.40,17.482873
5,660000,1150.0,90.0,2.40,9.166856
6,208000,NaN,NaN,NaN,191.009670
7,880000,1200.0,88.0,2.40,99.412159
8,620000,1020.0,90.0,NaN,3.169213
9,325000,NaN,NaN,NaN,17.573711


In [20]:
from sklearn.impute import KNNImputer
data_imputed = rank_data.values
ix = [i for i in range(data_imputed.shape[1]) if i != len(rank_data.columns)]
X = data_imputed[:, ix]
imputer = KNNImputer(n_neighbors=len(rank_data.columns))
imputer.fit(X)
Xtrans = imputer.transform(X)
X.shape

(90, 5)

In [21]:
rank_data = pd.DataFrame(Xtrans,index=rank_data.index, columns=rank_data.columns)
ra

In [22]:
import seaborn as sns
import matplotlib.pyplot as plt
def print_heatmap(cor_method):
  corrmat = rank_data.corr(method=cor_method)
  print("{} correlation heatmap\n".format(cor_method))
  f, ax = plt.subplots(figsize=(12, 9))
  sns.heatmap(corrmat,annot= True, vmax=.8, square=True)
  plt.show()
print_heatmap('pearson')
print_heatmap('spearman')

,price,pin(mAh),weight(g),screen_size(inch),rank_point
0,350000.0,800.0,73.0,1.770,261.982173
1,340000.0,800.0,73.0,1.770,222.593702
2,599000.0,1020.0,100.0,2.400,23.847440
3,590000.0,1020.0,90.0,2.400,105.041692
4,1150000.0,1500.0,96.4,2.400,17.482873
5,660000.0,1150.0,90.0,2.400,9.166856
6,208000.0,1020.0,136.4,1.896,191.009670
7,880000.0,1200.0,88.0,2.400,99.412159
8,620000.0,1020.0,90.0,2.280,3.169213
9,325000.0,1610.0,144.2,2.028,17.573711


In [23]:
from sklearn.preprocessing import StandardScaler

Standard_scaler = StandardScaler()
Xtrans = Standard_scaler.fit_transform(Xtrans)

In [24]:
rank_data_scaled = pd.DataFrame(Xtrans,index=rank_data.index, columns=rank_data.columns)

In [25]:
from skcriteria import Data, MIN, MAX
ranking = Data(
    rank_data_scaled,
    [MIN, MAX, MIN, MAX, MAX],
    cnames = rank_data.columns,
    weights = [1,5,5,1,10]
)

In [26]:
from skcriteria.madm import simple
dm = simple.WeightedSum(mnorm = "sum",wnorm = "sum")
dec = dm.decide(ranking)

In [27]:
rank_data['rank'] = dec.rank_
rank_data.sort_values(by=['rank'])

,price,pin(mAh),weight(g),screen_size(inch),rank_point,rank
0,350000.0,800.0,73.0,1.770,261.982173,1
1,340000.0,800.0,73.0,1.770,222.593702,2
6,208000.0,1020.0,136.4,1.896,191.009670,3
10,330000.0,1690.0,144.2,2.028,171.742887,4
3,590000.0,1020.0,90.0,2.400,105.041692,5
7,880000.0,1200.0,88.0,2.400,99.412159,6
12,720000.0,1020.0,103.0,2.400,65.870465,7
11,209000.0,1000.0,114.0,1.770,52.771730,8
13,460000.0,800.0,140.0,1.800,46.328056,9
47,199000.0,800.0,125.0,1.770,41.013265,10


In [28]:
cols_to_keep = ['index', 'name', 'brand', 'number_of_comment', 'Negative', 'Positive']
group_data = group_data.drop(group_data.columns.difference(cols_to_keep), axis=1)

In [30]:
pd.concat([group_data, rank_data], axis=1, join="inner").to_csv('normal_phone_done.csv')